---
# Stochastic Simulation Assignment 3
---

### **Contributors**  
- **Maarten Stork** - 15761770
- **Paul Jungnickel** - 15716554
- **Lucas Keijzer** - 14041073

### **Overview**  
This notebook contains the code and analysis for **Assignment 3 of Stochastic Simulation**. The code follows the order specified in the assignment guidelines and replicates the experiments conducted in the referenced paper. Each section corresponds to (a) key experiment(s).
The entire code required for running this notebook is contained in the repository https://github.com/PaulJungnickel/Stochastic_Simulation_Assignment_3

In [ ]:
from CircleParticleSim import *
from scipy.stats import mannwhitneyu
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

import numpy as np
import numpy.random as rand
import pandas as pd
import matplotlib.pyplot as plt

---

# 1) Optimal Particle Configuration

The following code block generates visual representations of the optimal configurations for \(n = 11, 12,\) and \(50\). These visualizations depict the best possible outcomes and have been included in the report.

In [ ]:
def run_experiment(steps, num_particles, num_runs=10):
    """
    Run the simulation for a given number of particles and store configurations and energies.

    Parameters:
    - steps: The number of simulation steps to perform.
    - num_particles: The number of particles in the simulation.
    - num_runs: The number of simulation runs to average results over.

    Returns:
    - results: A dictionary with averaged metrics (e.g., internal counts, total energy).
    - raw_data: A list of dictionaries containing detailed run information.
    - energy_examples: A dictionary storing particle locations and corresponding energies.
    """
    internal_counts = []
    energies = []
    raw_data = []
    energy_examples = {"locations": [], "energies": []}

    # Initialize and run the simulation
    for run in range(num_runs):
        sim = CircleParticleSim(
            N=num_particles,
            cooling_schedule=paper_cooling_schedule,
            step_size_schedule=sqrt_step_size_schedule,
            steps=steps
        )
        sim.run_simulation(steps)

        # Calculate floating particles (not on the edge)d
        floating_count = np.sum(np.linalg.norm(sim.particle_locations, axis=1) < 0.99)
        internal_counts.append(floating_count)
        energies.append(sim.E)

        # Store raw data
        raw_data.append({
            "Run": run,
            "Floating Count": floating_count,
            "Energy": sim.E
        })

        # Save configurations and energies
        energy_examples["locations"].append(sim.particle_locations)
        energy_examples["energies"].append(sim.E)

    # Calculate metrics for all runs
    avg_internal_count = np.mean(internal_counts)
    min_internal_count = np.min(internal_counts)
    max_internal_count = np.max(internal_counts)

    # Compile results into a dictionary
    results = {
        "Particles": num_particles,
        "Internal Count (Avg)": avg_internal_count,
        "Internal Count (Min)": min_internal_count,
        "Internal Count (Max)": max_internal_count,
        "Total Energy (Avg)": np.mean(energies)
    }

    print(f"Completed simulation for {num_particles} particles (averaged over {num_runs} runs).")

    return results, raw_data, energy_examples

def plot_selected_configurations(selected_examples):
    """
    Plot configurations for n=11, n=12, and n=50, each showing the lowest energy configuration.

    Parameters:
    - selected_examples: A dictionary containing particle locations and energies for each particle count.
    """
    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 24))  # Adjust layout for vertical alignment
    particle_counts = [11, 12, 50]

    for i, (ax, n_particles) in enumerate(zip(axes, particle_counts)):
        # Retrieve locations and energies for the current particle count
        locations = selected_examples[n_particles]["locations"]
        energies = selected_examples[n_particles]["energies"]

        # Find the best configuration (lowest energy)
        best_index = np.argmin(energies)
        best_configuration = locations[best_index]
        best_energy = energies[best_index]

        # Plot the configuration
        thetas = np.linspace(0, 2 * np.pi, 100)
        ax.plot(np.cos(thetas), np.sin(thetas), linestyle='--', color='gray', linewidth=2, alpha=0.7)
        ax.scatter(
            best_configuration[:, 0], best_configuration[:, 1],
            color='blue', edgecolor='black', s=200, alpha=0.9
        )

        # Set the title
        ax.set_title(
            f"n={n_particles}\nLowest Energy: {best_energy:.2f}",
            fontsize=24, fontweight='bold', color='navy', pad=30
        )

        # Styling
        ax.set_xlim([-1.2, 1.2])
        ax.set_ylim([-1.2, 1.2])
        ax.set_aspect('equal')
        ax.axis('off')

    # Add a global caption
    plt.figtext(
        0.5, 0.01,
        "Selected Optimal Configurations: n=11, n=12, n=50 (lowest energy for each).\n"
        "Simulated annealing used to minimize energy and arrange particles within a circular boundary.",
        ha='center', fontsize=18, color='darkgray', wrap=True
    )

    plt.tight_layout(rect=[0, 0.05, 1, 0.95])
    plt.show()

def create_and_save_node_energy_table(results, output_file):
    """
    Create a DataFrame for particle counts and their optimal metrics, and save it to a CSV file.

    Parameters:
    - results: The results data collected from the simulation.
    - output_file: Path to save the CSV file.
    """
    df = pd.DataFrame(results)
    print("Node Count and Optimal Metrics Table:")
    print(df)

    # Save to a CSV file
    df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")
    return df

if __name__ == '__main__':
    step_counts = 100000
    num_runs = 50
    particle_counts = [11, 12, 50]

    selected_examples = {}

    # Run experiments for the selected particle counts
    for n in particle_counts:
        print(f"Running experiment for n={n}...")
        results, raw_data, energy_examples = run_experiment(
            steps=step_counts, num_particles=n, num_runs=num_runs
        )
        selected_examples[n] = energy_examples

    # Plot the selected configurations
    plot_selected_configurations(selected_examples)


The following code block runs the model for particle counts ranging from \(n = 9\) to \(n = 50\). It gathers data on particle counts, optimal energies, optimal positional configurations, and the proportional frequency of observing these optimal configurations in the results. A table summarizing these findings has been included in the paper.


---
# 2) Cooling Schedules

This block defines the experimental parameters (e.g., number of particles, steps, and runs) and the cooling schedules to be evaluated.

In [ ]:
# Parameters Used:
num_particles = 12
steps = 10000
num_runs = 50


# Schedules Used:
schedules = [
# log_cooling_schedule,
# basic_cooling_schedule,
paper_cooling_schedule,
exponential_cooling_schedule,
# linear_cooling_schedule,
# quadratic_cooling_schedule,
sigmoid_cooling_schedule,
inverse_sqrt_cooling_schedule,
cosine_annealing_cooling_schedule,
# stepwise_cooling_schedule,
]

This next block runs simulations for each cooling schedule, evaluates the mean energy, standard deviation of energy, and mean temperature across multiple runs, and saves the results to a `.npy` file for later analysis.

In [ ]:
data = {}

# generate data
for schedule in schedules:
    print(f"Currently running for: {schedule.__name__}")
    mean_energy, std_energy, mean_temperatures = evaluate_multiple_runs(
        num_particles, cooling_schedule=schedule, steps=steps, num_runs=num_runs
    )
    data[schedule.__name__] = {
        "mean_energy": mean_energy,
        "std_energy": std_energy,
        "mean_temperatures": mean_temperatures
    }

# Save data
np.save('data/data-2-{}-{}.npy'.format(num_particles, len(schedules)), data)


This block loads the saved data from the previous block, plots the temperature and energy evolution over time for each schedule, and highlights the standard deviation. It also includes a zoomed-in inset to visualize the energy behavior in detail over specific ranges of steps.

In [ ]:
loaded_data = np.load('data/data-2-{}-{}.npy'.format(num_particles, len(schedules)), allow_pickle=True).item()

fig, axs = plt.subplots(2, 1, figsize=(10, 12), sharex=True)

axins = inset_axes(
    axs[1],
    width="30%", 
    height="50%", 
    loc='upper right'
)

for schedule_name, results in loaded_data.items():
    mean_energy = results["mean_energy"]
    std_energy = results["std_energy"]
    mean_temperatures = results["mean_temperatures"]
    print(f"min energy: {min(mean_energy)} for schedule: {schedule_name}")
    axs[1].plot(mean_energy, label=schedule_name)
    axs[1].fill_between(
        range(len(mean_energy)),
        mean_energy - std_energy,
        mean_energy + std_energy,
        alpha=0.3
    )
    axs[0].plot(mean_temperatures, label=schedule_name)
    
    axins.plot(mean_energy, label=schedule_name)
    axins.fill_between(
        range(len(mean_energy)),
        mean_energy - std_energy,
        mean_energy + std_energy,
        alpha=0.3
    )

# Plot for temperature
axs[0].set_xlabel("Steps")
axs[0].set_ylabel("Temperature")
axs[0].set_title("Temperature Evolution Over Time")
axs[0].set_xscale('log')
# axs[1].set_yscale('log')
axs[0].legend()
axs[0].grid(True)

# Plot for energy
axs[1].set_ylabel("Energy")
axs[1].set_title("Mean Energy with Standard Deviation Over Time")
axs[1].set_xscale('log')
# axs[1].set_yscale('log')
# axs[1].legend()
axs[1].grid(True)


skip_first_steps = 0
plt.xlim(left=skip_first_steps)

# zoomed in plot
axins.set_xlim(10**2, 10**4)
axins.set_ylim(119, 121)
axins.set_xscale('log')
axins.set_yscale('log')
axins.grid(True)

mark_inset(axs[1], axins, loc1=2, loc2=4, fc="none", ec="0.5")




plt.tight_layout()
plt.show()

## Statistical comparison

Below some code is written to compare the final step of each schedule

In [ ]:
loaded_data = np.load('data/data-2-{}-{}.npy'.format(num_particles, len(schedules)), allow_pickle=True).item()

schedule_names = [
"Paper",
"Exponential",
"Sigmoid",
"Inverse sqrt",
"Cosine annealing",
]


final_mean_energies = []
final_std_energies = []
labels = []

for schedule_name, results in loaded_data.items():
    mean_energy = results["mean_energy"]
    std_energy = results["std_energy"]
    final_mean_energies.append(mean_energy[-1])
    final_std_energies.append(std_energy[-1])
    labels.append(schedule_name)

plt.figure(figsize=(6, 6))

# Plot a simple scatter with error bars, since we only have mean and std:
plt.errorbar(schedule_names, final_mean_energies, yerr=final_std_energies, fmt='o', capsize=10, linewidth=2, elinewidth= 2)
plt.xticks(rotation=45, ha='right', fontsize=14)
plt.ylabel('Final Energy', fontsize=14)
plt.tight_layout()

plt.savefig('plots/last_step', dpi=600)

plt.show()


In [ ]:
labels = []
final_energies = []

for schedule_name, stats in loaded_data.items():
    energies = stats['energies'][:, -1]  # all runs at final step
    labels.append(schedule_name)
    final_energies.append(energies)

# Determine subplot arrangement
n_schedules = len(labels)
cols = 2
rows = int(np.ceil(n_schedules / cols))

fig, axes = plt.subplots(rows, cols, figsize=(12, 4 * rows))
axes = axes.flatten() if n_schedules > 1 else [axes]

# Plot a histogram for each schedule
bins = 20  
for i, (label, energies) in enumerate(zip(labels, final_energies)):
    ax = axes[i]
    ax.hist(energies, bins=bins, color='skyblue', edgecolor='black', alpha=0.7)
    ax.set_title(label, fontsize=14)
    ax.set_xlabel('Final Step Energy')
    ax.set_ylabel('Frequency')
    ax.grid(axis='y', linestyle='--', alpha=0.7)

# If there are fewer schedules than the subplot slots, hide extra axes
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Compare the first schedule with all the others
first_schedule_label = labels[0]
first_schedule_energies = final_energies[0]

print(f"Mann-Whitney U Test results comparing '{first_schedule_label}' to others:")
for label, energies in zip(labels[1:], final_energies[1:]):
    # Perform the Mann-Whitney U test
    stat, p_value = mannwhitneyu(first_schedule_energies, energies, alternative='two-sided')
    
    print(f"Comparison with {label}: U-statistic={stat}, p-value={p_value}")


---
# 3) Chain Length

In the next block, simulations are conducted for different chain length scaling factors, with energies recorded for 100 runs at each scaling. Results are saved for analysis.

In [ ]:
rand.seed(42)
run_count = 20
num_particles = 50
num_scales = 15
chain_lens = np.int64(np.round(np.logspace(1,4,num=num_scales, base=10)))
max_cooling_steps = 1000
print(chain_lens)

def run_scales(num_particles):
    arrs = np.zeros([num_scales, run_count]) 

    for i, chain_len in enumerate(chain_lens):
        num_steps = chain_len * max_cooling_steps
        # cooling_steps =  int(num_steps/chain_len)
        for j in range(run_count):
            sim  = CircleParticleSim(num_particles, steps=num_steps, seed=rand.randint(0,2**31-1),
                        cooling_schedule = paper_cooling_schedule,
                        step_size_schedule = sqrt_step_size_schedule,
                        random_step_likelihood=0.2,
                        )
            sim.run_simulation(max_cooling_steps, chain_len)
            arrs[i,j] = sim.E

        print(sim.T)

        
    np.save('data/data-3.1-{}-{}.npy'.format(num_particles, num_scales), arrs)

# for num_particles in [22,50,80]:
#     run_scales(num_particles)

num_scales = 15
arrs = np.load('data/data-3.1-50-{}.npy'.format(num_scales))
arrs = arrs[:]
scales = np.logspace(-2,1,num=num_scales, base=10)
scales1 = 100*scales[:]
scales1 = np.int64(np.round(np.logspace(1,4,num=num_scales, base=10)))
print(arrs.shape)
min_energy = np.min(arrs)
arrs -= min_energy
mean = np.mean(arrs, axis=1)
min_energy = 0


fig, ax1 = plt.subplots(figsize = (5.5,5), layout='constrained')

ax2 = ax1.twinx()

p1 = ax1.plot(scales1, mean, label='mean and range of final energies')
perc = 1
ax1.fill_between(scales1, np.percentile(arrs, perc, axis=1), np.percentile(arrs, 100-perc, axis=1), alpha=0.3)


opt_count = np.count_nonzero((np.abs(arrs - min_energy) < 1e-3), axis=1) / 20
p2 = ax2.plot(scales1, opt_count, linestyle='', marker='s', c='tab:orange', label='share of runs that reach the global minimum')
ax1.set_xlabel('Markov Chain Length', fontsize=14)
ax1.set_ylabel(r'$E - E_{min}$', fontsize=14)
ax1.set_yscale('log')
ax1.set_ylim([1e-6-1e12, 1e2-1e-5])
ax2.set_ylim([0,0.99])
plt.xscale('log')
ax1.yaxis.set_tick_params(labelsize=12, labelcolor='#235b7c') 
ax2.yaxis.set_tick_params(labelsize=12, labelcolor='#E96700') 
ax1.xaxis.set_tick_params(labelsize=12, ) 
ax1.legend(handles=p1+p2, loc='upper center')

# fig.savefig('plots/3-50nodes-fin.pdf')

---

# 4) Neighboring

## Direction Strategy

In the next block, simulations are performed for different probabilities of random step selection. Final energies are recorded and saved for analysis.

In [ ]:
rand.seed(42)
run_count = 50
num_probs = 20
num_particles = 50
# probs = np.linspace(0,1,num_probs)

chain_len = 1000
max_cooling_steps = 300
num_steps = chain_len * max_cooling_steps

step_size_scheds = [const_step_size_schedule, random_step_size_schedule, sqrt_step_size_schedule]

def run_probs(num_particles):
    arrs = np.zeros([num_probs, run_count]) 

    for i, p in enumerate(probs):
        for j in range(run_count):
            sim  = CircleParticleSim(num_particles, steps=num_steps, seed=rand.randint(0,2**31-1),
                        cooling_schedule = paper_cooling_schedule,
                        step_size_schedule = sqrt_step_size_schedule,
                        random_step_likelihood=p
                        )
            sim.run_simulation(max_cooling_steps, chain_len)
            arrs[i,j] = sim.E

        print('.', end='')

        
    np.save('data/data-4.1-{}-{}.npy'.format(num_particles, num_probs), arrs)


# for num_particles in [30, 80, 100]:
#     run_probs(num_particles)

fig, axs = plt.subplots(1,1,sharex=True, figsize=(5,5),layout='constrained')
for i, num_particles in enumerate([22, 50,  80]):
    arrs = np.load('data/data-4.1-{}-20.npy'.format(num_particles))
    arrs = arrs[:]/2
    num_probs = 20
    probs = np.linspace(0,1,num_probs)
    probs1 = probs[:]
    min_energy = np.min(arrs)
    arrs = arrs -  min_energy
    min_energy=0
    print(arrs.shape)
    mean = np.mean(arrs, axis=1)
    merr = 1.96 / np.sqrt(run_count) *np.std(arrs, axis=1)
    axs.errorbar(probs1, mean, merr, linestyle='', marker='o', label='{} particles'.format(num_particles), alpha=0.8)
    axs.set_yscale('log')
plt.xlabel('P(random direction)', fontsize=14)
plt.ylabel(r'$E - E_{min}$', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend()
# plt.savefig('plots/4-fin.pdf')



## Step Size Schedules Comparison

In [ ]:
rand.seed(42)
run_count = 50
num_probs = 20
num_particles = 50
probs = np.linspace(0,1,num_probs)

chain_len = 1000
max_cooling_steps = 1000
num_steps = chain_len * max_cooling_steps

step_size_scheds = [const_step_size_schedule, random_step_size_schedule, sqrt_const_step_size_schedule, sqrt_step_size_schedule]

def run_single(step_schedule):
    arrs = np.zeros([num_probs, run_count]) 

    for i, p in enumerate(probs):
        for j in range(run_count):
            sim  = CircleParticleSim(num_particles, steps=num_steps, seed=rand.randint(0,2**31-1),
                        cooling_schedule = paper_cooling_schedule,
                        step_size_schedule = step_schedule,
                        random_step_likelihood=0.2
                        )
            sim.run_simulation(max_cooling_steps, chain_len)
            arrs[i,j] = sim.E

        print('.', end='')

        
    np.save('data/data-4.2-{}-{}.npy'.format(num_particles, num_probs), arrs)


# for step_schedule in step_size_scheds:
#     run_probs(step_schedule)


fig, axs = plt.subplots(1,1,sharex=True, figsize=(5,5),layout='constrained')
data = []
swap = [0,2,1,3]
for i, step_schedule in enumerate(step_size_scheds[0:]):
    arrs = np.load('data/data-4.3-{}.npy'.format(swap[i]))[0]
    arrs = arrs/2
    arrs = arrs -  1459.5821
    data.append(arrs)

labels = ['0.1',  r'$\sqrt{T}$', 'U(0,1)',r'$U(0,1)\sqrt{T}$']

bplot = plt.boxplot(data, patch_artist=True, tick_labels=labels)
plt.yscale('log')
plt.xlabel('step size r', fontsize=14)
plt.ylabel(r'$E - E_{min}$', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('plots/4-steps.pdf')

for i in range(4):
    for j in range(i+1, 4):
        print(i, j, scipy.stats.mannwhitneyu(data[i], data[j]))